# 🧠 Waste Classification & Detection API
API này sử dụng FastAPI để phân loại rác và phát hiện đối tượng bằng TensorFlow & YOLOv8.
## 🗂️ Nội dung:
- Tải mô hình TensorFlow & YOLO
- Phân loại rác (Tái chế / Không tái chế)
- Phân loại chi tiết loại rác
- Phát hiện đối tượng rác bằng YOLOv8
- FastAPI Endpoint & hướng dẫn chạy


In [ ]:
# 🔧 1. Import thư viện và cấu hình hệ thống
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from ultralytics import YOLO
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List, Dict
import logging

## 📁 2. Đường dẫn mô hình và khởi tạo biến toàn cục

In [ ]:
MODEL_PATHS = {
    'recyclable': 'models/Model1.Best.keras',
    'recyclable_detail': 'models/model2A_EfficientNetB2.keras',
    'non_recyclable_detail': 'models/model2B_EfficientNetB2_best.keras',
    'yolo': 'runs/detect/train4/weights/last.pt'
}
models = {}  # chứa các model đã load

## 🧠 3. Hàm load model

In [ ]:
def load_models():
    for name, path in MODEL_PATHS.items():
        if name == 'yolo':
            models[name] = YOLO(path)
        else:
            models[name] = tf.keras.models.load_model(path, compile=False)
    print("✅ Models loaded!")

## 🖼️ 4. Tiền xử lý ảnh trước khi phân loại

In [ ]:
def preprocess_image(image_np, target_size=(224, 224)):
    img = cv2.resize(image_np, target_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype('float32') / 255.0
    return np.expand_dims(img, axis=0)

## ♻️ 5. Phân loại rác: Tái chế hay không? Sau đó chi tiết loại rác

In [ ]:
def classify_garbage(image_np):
    processed = preprocess_image(image_np)
    prob = models['recyclable'].predict(processed)[0][0]
    if prob > 0.5:
        detail = models['recyclable_detail'].predict(processed)[0]
        label = np.argmax(detail)
        return 'Tái chế', label, prob
    else:
        detail = models['non_recyclable_detail'].predict(processed)[0]
        label = np.argmax(detail)
        return 'Không tái chế', label, 1 - prob

## 📦 6. Phát hiện rác bằng YOLOv8

In [ ]:
def detect_yolo(image_pil):
    result = models['yolo'](image_pil)[0]
    for box in result.boxes:
        print(models['yolo'].names[int(box.cls[0])], box.xyxy[0].tolist())